In [1]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

# List of tools
tools = [add, multiply]


In [2]:
from langchain_ollama import ChatOllama

# Configure the Ollama model
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0  # Deterministic response
)

# Bind tools to the model
llm_with_tools = llm.bind_tools(tools)


In [ ]:
# Query the model
response = llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations."
)

# Print the tool calls generated by the model
print("Tool Calls:", response.tool_calls)


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Few-shot examples
examples = [
    HumanMessage("What's the product of 317253 and 128472 plus four", name="example_user"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "Multiply", "args": {"a": 317253, "b": 128472}, "id": "1"}]),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "Add", "args": {"a": 16505054784, "b": 4}, "id": "2"}]),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage("The product of 317253 and 128472 plus four is 16505054788", name="example_assistant"),
]

# System message and few-shot prompt
system_message = """You are bad at math but are an expert at using a calculator. 
Use past tool usage as an example of how to correctly use the tools."""

few_shot_prompt = ChatPromptTemplate.from_messages(
    [("system", system_message), *examples, ("human", "{query}")]
)


In [ ]:
# Chain to include few-shot prompting
chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools

# Query the chain
response = chain.invoke("Whats 119 times 8 minus 20")
print("Tool Calls:", response.tool_calls)
